In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118                                                                                                                                                                                                                    

In [ ]:
%pip install opencv-python pillow

In [ ]:
%pip install basicsr==1.4.2
%pip install realesrgan==0.3.0
%pip install torchvision==0.16.2
%pip install numpy==1.26.4

In [ ]:
import numpy as np
import torch
import torchvision
from realesrgan.utils import RealESRGANer


print("NumPy:", np.__version__)
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
import os
import cv2
import torch

from realesrgan.utils import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet

# =====================================================
# PROJECT ROOT (auto-detects current folder)
# =====================================================
PROJECT_ROOT = os.getcwd()

MODEL_PATH = os.path.join(PROJECT_ROOT, "models", "RealESRGAN_x4plus.pth")
INPUT_IMAGE = os.path.join(PROJECT_ROOT, "input", "test.jpg")
OUTPUT_IMAGE = os.path.join(PROJECT_ROOT, "output", "test_x4.png")

# =====================================================
# DEVICE CHECK
# =====================================================
if not torch.cuda.is_available():
    raise RuntimeError("CUDA not available. GPU is NOT being used.")

DEVICE = "cuda"
print("✅ GPU:", torch.cuda.get_device_name(0))

# =====================================================
# FILE CHECKS
# =====================================================
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model not found: {MODEL_PATH}")

if not os.path.exists(INPUT_IMAGE):
    raise FileNotFoundError(f"Input image not found: {INPUT_IMAGE}")

os.makedirs(os.path.dirname(OUTPUT_IMAGE), exist_ok=True)

print("✅ Files verified")

# =====================================================
# MODEL DEFINITION (Real-ESRGAN x4)
# =====================================================
model = RRDBNet(
    num_in_ch=3,
    num_out_ch=3,
    num_feat=64,
    num_block=23,
    num_grow_ch=32,
    scale=4
)

upsampler = RealESRGANer(
    scale=4,
    model_path=MODEL_PATH,
    model=model,
    tile=512,        # Safe for GTX 1650
    tile_pad=10,
    pre_pad=0,
    half=False,      # 🚨 IMPORTANT FIX
    device=DEVICE
)

# =====================================================
# LOAD IMAGE
# =====================================================
img = cv2.imread(INPUT_IMAGE)

if img is None:
    raise RuntimeError("Failed to read image (corrupted or unsupported format).")

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print("🚀 Upscaling started...")

# =====================================================
# UPSCALE
# =====================================================
output, _ = upsampler.enhance(img, outscale=4)

# =====================================================
# SAVE OUTPUT
# =====================================================
output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
cv2.imwrite(OUTPUT_IMAGE, output)

print("✅ Upscaling complete")
print("📁 Output saved to:", OUTPUT_IMAGE)
